In [2]:
import cv2
import pyautogui
import mediapipe as mp
import time

In [6]:
cap = cv2.VideoCapture(0)

mp_hand = mp.solutions.hands

myhand = mp_hand.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

while True:
    ret, frame = cap.read()

    if not ret: break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = myhand.process(image_rgb)

    if results.multi_hand_landmarks:
        for handlandmark in results.multi_hand_landmarks:
            print(handlandmark)
            mp_drawing.draw_landmarks(frame, handlandmark, mp_hand.HAND_CONNECTIONS)

            thumb_tip = handlandmark.landmark[mp_hand.HandLandmark.THUMB_TIP]
            index_finger_tip = handlandmark.landmark[mp_hand.HandLandmark.INDEX_FINGER_TIP]
            middle_finger_tip = handlandmark.landmark[mp_hand.HandLandmark.MIDDLE_FINGER_TIP]
            ring_finger_tip = handlandmark.landmark[mp_hand.HandLandmark.RING_FINGER_TIP]
            pinky_tip = handlandmark.landmark[mp_hand.HandLandmark.PINKY_TIP]

            is_hand_close = (
                index_finger_tip.y > thumb_tip.y and 
                middle_finger_tip.y > thumb_tip.y and
                ring_finger_tip.y > thumb_tip.y and 
                pinky_tip.y > thumb_tip.y
            )

            if is_hand_close:
                pass
            else:
                pyautogui.press('space')
                time.sleep(0.1)



    cv2.imshow('CV', cv2.flip(frame,1))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

In [1]:
import cv2
import mediapipe as mp
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import math

In [3]:

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volume.GetMute()
volume.GetMasterVolumeLevel()
volumeRange = volume.GetVolumeRange()
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
cap = cv2.VideoCapture(0)

# Class to represent a draggable rectangle
class DragRect():
    def __init__(self, posOrigin, size=[100, 100]):
        self.posOrigin = posOrigin
        self.size = size
        self.dragging = False
        self.offsetX = 0
        self.offsetY = 0

    def update(self, cursor):
        ox, oy = self.posOrigin
        w, h = self.size

        # Check if cursor is inside the rectangle
        if ox < cursor[0] < ox + w and oy < cursor[1] < oy + h:
            self.dragging = True
            self.offsetX = cursor[0] - ox
            self.offsetY = cursor[1] - oy

        # Move the rectangle if dragging
        if self.dragging:
            self.posOrigin = cursor[0] - self.offsetX, cursor[1] - self.offsetY

    def draw(self, img):
        ox, oy = self.posOrigin
        w, h = self.size
        cv2.rectangle(img, (ox, oy), (ox + w, oy + h), (0, 255, 0), 2)

# Create a draggable rectangle
rect = DragRect([100, 100], [200, 200])

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Extract landmarks for index finger tip and middle finger tip
        index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
        
        # Calculate distance between index finger tip and middle finger tip
        distance = math.sqrt((index_tip.x - middle_tip.x)**2 + (index_tip.y - middle_tip.y)**2)
        
        # If distance is long, update rectangle position
        if distance > 0.1:  # Adjust threshold as needed
            cursor = int(index_tip.x * image.shape[1]), int(index_tip.y * image.shape[0])
            rect.update(cursor)

    # Draw the draggable rectangle
    rect.draw(image)

    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()


In [7]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import numpy as np

In [11]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
detector = HandDetector(detectionCon=0.8)
colorR = (255, 0, 255)

cx, cy, w, h = 100, 100, 200, 200

class DragRect():
    def __init__(self, posCenter, size=[200, 200]):
        self.posCenter = posCenter
        self.size = size

    def update(self, cursor):
        cx, cy = self.posCenter
        w, h = self.size

        if cx - w // 2 < cursor[0] < cx + w // 2 and \
                cy - h // 2 < cursor[1] < cy + h // 2:
            self.posCenter = cursor

rectList = []
for x in range(5):
    rectList.append(DragRect([x * 250 + 150, 150]))

while True:
    ret, img = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame from the webcam")
        break

    img = cv2.flip(img, 1)
    img = detector.findHands(img)
    lmList, _ = detector.findPosition(img)
    

    if lmList:
        l, _, _ = detector.findDistance(8,12, img, draw=False)
        print("Distance:", l)
        
        if l < 30:
            cursor = lmList[8]
            for rect in rectList:
                rect.update(cursor)

    imgNew = np.zeros_like(img, np.uint8)
    for rect in rectList:
        cx, cy = rect.posCenter
        w, h = rect.size
        cv2.rectangle(imgNew, (cx - w // 2, cy - h // 2),
                      (cx + w // 2, cy + h // 2), colorR, cv2.FILLED)
        cvzone.cornerRect(imgNew, (cx - w // 2, cy - h // 2, w, h), 20, rt=0)

    out = img.copy()
    alpha = 0.5
    mask = imgNew.astype(bool)
    out[mask] = cv2.addWeighted(img, alpha, imgNew, 1 - alpha, 0)[mask]

    cv2.imshow("Image", out)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'HandDetector' object has no attribute 'findPosition'

In [1]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime
 
video_capture = cv2.VideoCapture(0)
 
jobs_image = face_recognition.load_image_file("photos/jobs.jpg")
jobs_encoding = face_recognition.face_encodings(jobs_image)[0]
 
ratan_tata_image = face_recognition.load_image_file("photos/tata.jpg")
ratan_tata_encoding = face_recognition.face_encodings(ratan_tata_image)[0]
 
sadmona_image = face_recognition.load_image_file("photos/sadmona.jpg")
sadmona_encoding = face_recognition.face_encodings(sadmona_image)[0]
 
tesla_image = face_recognition.load_image_file("photos/tesla.jpg")
tesla_encoding = face_recognition.face_encodings(tesla_image)[0]
 
known_face_encoding = [
jobs_encoding,
ratan_tata_encoding,
sadmona_encoding,
tesla_encoding
]
 
known_faces_names = [
"jobs",
"ratan tata",
"sadmona",
"tesla"
]
 
students = known_faces_names.copy()
 
face_locations = []
face_encodings = []
face_names = []
s=True
 
 
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
 
 
 
f = open(current_date+'.csv','w+',newline = '')
lnwriter = csv.writer(f)
 
while True:
    _,frame = video_capture.read()
    small_frame = cv2.resize(frame,(0,0),fx=0.25,fy=0.25)
    rgb_small_frame = small_frame[:,:,::-1]
    if s:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame,face_locations)
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encoding,face_encoding)
            name=""
            face_distance = face_recognition.face_distance(known_face_encoding,face_encoding)
            best_match_index = np.argmin(face_distance)
            if matches[best_match_index]:
                name = known_faces_names[best_match_index]
 
            face_names.append(name)
            if name in known_faces_names:
                font = cv2.FONT_HERSHEY_SIMPLEX
                bottomLeftCornerOfText = (10,100)
                fontScale              = 1.5
                fontColor              = (255,0,0)
                thickness              = 3
                lineType               = 2
 
                cv2.putText(frame,name+' Present', 
                    bottomLeftCornerOfText, 
                    font, 
                    fontScale,
                    fontColor,
                    thickness,
                    lineType)

                if name in students:
                    students.remove(name)
                    print(students)
                    current_time = now.strftime("%H-%M-%S")
                    lnwriter.writerow([name,current_time])
    cv2.imshow("attendence system",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
video_capture.release()
cv2.destroyAllWindows()
f.close()

ModuleNotFoundError: No module named 'face_recognition'

In [2]:
%pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached dlib-19.24.4.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Failed to build dlib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [13 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Traceback (most recent call last):
        File "D:\education\BFCAI 3rd\2nd Term\Neural Network\Sections\Coding\lib\runpy.py", line 197, in _run_module_as_main
          return _run_code(code, main_globals, None,
        File "D:\education\BFCAI 3rd\2nd Term\Neural Network\Sections\Coding\lib\runpy.py", line 87, in _run_code
          exec(code, run_globals)
        File "D:\education\BFCAI 3rd\2nd Term\Neural Network\Sections\Coding\Scripts\cmake.exe\__main__.py", line 4, in <module>
      ModuleNotFoundError: No module named 'cmake'
      
      ERROR: CMake must be installed to build dlib
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for dlib
ERROR: Cou